# **PolyCoder**
zdroj: https://github.com/VHellendoorn/Code-LMs

Tři různé úrovně modulů:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    -PolyCoder-160M  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    -PolyCoder-0.4B  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    -PolyCoder-2.7B  

V této práci je využit model PolyCoder-2.7B  
Modely byly trénovány na velkém datasetu kódu zahrnujícího 12 programovacích jazyků. To zahrnuje 2,7B parametrický model (přezdívaný PolyCoder , trénovaný pro 100K a 150K kroků), 405M parametrický model (100K & 150K kroků) a 160M parametrický model (150K kroků).

#### **Datové sady**
249GB vícejazyčný dataset. Seznam cest k souborům lze stáhnout z: https://zenodo.org/record/6363556/files/index.zip . Každý řádek v souboru je cesta k souboru spolu s jeho SHA-256 hash, aby se usnadnila deduplikace. To znamená, že hash umožňuje kontrolu, zda soubory z nějaké budoucí testovací sady již byly obsaženy v trénovací sadě.

Proces sběru a filtrování dat je podrobně popsán v článku a níže. Konečné, filtrované statistiky datové sady jsou: 
   
![dataset](./dataset.png)  
  
**Sběr a filtrování dat**

V říjnu 2021 byly z GitHubu naklonovány nejoblíbenější úložiště pro 12 oblíbených programovacích jazyků s alespoň 50 hvězdičkami. U každého projektu byl extrahován každý soubor patřící do většinového jazyka tohoto projektu, čímž se získal tréninková sada níže (po vyčištění). Tato počáteční, nefiltrovaná datová sada zahrnovala 631 GB a 38,9 milionů souborů.

Dále, podobně jako u Codex a CodeParrot, byly odfiltrovány velmi velké (>1 MB) a velmi krátké (<100 tokenů) soubory, čímž se datová sada zmenšila na 424 GB. Soubory byly poté deduplikovány na základě hashe jejich obsahu, což snížilo počet souborů o dalších přibližně 30 %, takže zůstalo 249 GB dat a 24,1 milionu souborů. Nebyly použity žádné tokenizační filtry; model zpracovává celé soubory včetně všech komentářů.

#### **Hodnoceni**
![image.png](./HumanEval.png)

## **Implementace řešení**
Ověření dostupnosti grafické karty

In [4]:
import torch
print(torch.__version__)

# Kontrola, zda je dostupné CUDA (což znamená, že máte GPU)
if torch.cuda.is_available():  
  device = torch.device("cuda")
  print('Running on GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('Running on CPU')

Running on GPU: NVIDIA GeForce GTX 1650 with Max-Q Design


Využití knihovny Transformers, která obsahuje tisíce různých předtrénovaných modelů z Hugging Face Hub

In [8]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.23.0")

Z Hugging Face Hub stahuji tokenizer pro PolyCoder2.7B. Tokenizer je nástroj který umí rozdělit vstupní text na jednotlivé, pro model logické, tokeny  
  
Model PolyCoder2.7B je předtrenované strojevé učení pro generování programovacího kódu

In [9]:
tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-2.7B")
model = AutoModelForCausalLM.from_pretrained("NinedayWang/PolyCoder-2.7B")

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.26s/it]


In [11]:
# Přesunutí modelu na GPU
model = model.to(device)

def return1():\n  """Returns 1."""\n
def convert_to_number(string):
def binarySearch(arr, left, right, x):
    mid = (left +

<span style="color:lightblue">**input_ids:**</span> Představuje vstupní data pro model. Jsou to tokeny, které byly zakódovány pomocí tokenizeru.  
  
<span style="color:lightblue">**max_length:**</span> Maximální délka generovaného textu. Pokud je dosaženo této délky, generování se zastaví.  
  
<span style="color:lightblue">**num_beams:**</span> Počet paprsků použité při paprskovém vyhledávání. Paprskové vyhledávání je technika používaná v autoregresivním generování textu pro nalezení nejpravděpodobnějších výstupních sekvencí.  
  
<span style="color:lightblue">**num_return_sequences:**</span> Počet sekvencí, které mají být vráceny. Každá sekvence je generována nezávisle na ostatních.  
  
<span style="color:lightblue">**attention_mask:**</span> Maska, která určuje, na které tokeny by měl model věnovat pozornost1. Hodnota 1 znamená, že model by měl věnovat pozornost danému tokenu, zatímco hodnota 0 znamená, že by měl daný token ignorovat.  
  
<span style="color:lightblue">**pad_token_id:**</span> Určuje ID tokenu, který se používá pro doplnění (padding) sekvencí. Pokud je sekvence kratší než max_length, pak se použije tento token pro doplnění sekvence na max_length.  

In [15]:
prompt = '''public int getTotalWeight(List<Integer> weights) {\n\t\t// Sum weights in parallel.\n\t\treturn '''
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Vytvoření attention mask
attention_mask = torch.ones(input_ids.shape, device=device)

# Přesunutí vstupních dat na GPU
input_ids = input_ids.to(device)

result = model.generate(input_ids, max_length=150, num_beams=15, num_return_sequences=4, attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id)
for res in result:
    print(tokenizer.decode(res))

public int getTotalWeight(List<Integer> weights) {
		// Sum weights in parallel.
		return                                                                                                                            
public int getTotalWeight(List<Integer> weights) {
		// Sum weights in parallel.
		return                                                                                                                            

public int getTotalWeight(List<Integer> weights) {
		// Sum weights in parallel.
		return                                                                                                                            //
public int getTotalWeight(List<Integer> weights) {
		// Sum weights in parallel.
		return                                                                                                                           
		


In [6]:
prompt = '''def symmetric_encryption(key,plaintext):\n """Returns ciphertext"""'''
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Přesunutí vstupních dat na GPU
input_ids = input_ids.to(device)

result = model.generate(input_ids, max_length=50, num_beams=4, num_return_sequences=4)
for res in result:
    print(tokenizer.decode(res))

print("end")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def symmetric_encryption(key,plaintext):
 """Returns ciphertext"""
 if len(plaintext)!= len(key):
    raise ValueError("plaintext and key must be of the same length")
 if len(plaintext) == 0
def symmetric_encryption(key,plaintext):
 """Returns ciphertext"""
 if len(plaintext)!= len(key):
    raise ValueError("plaintext and key must be of the same length")
 if len(key)!= 16
def symmetric_encryption(key,plaintext):
 """Returns ciphertext"""
 if len(plaintext)!= len(key):
    raise ValueError("plaintext and key must be of the same length")
 if len(plaintext)!= len
def symmetric_encryption(key,plaintext):
 """Returns ciphertext"""
 if len(plaintext)!= len(key):
    raise ValueError("plaintext and key must be of the same length")
 if len(key)!= 8
end
